In [1]:
import pandas as pd
import numpy as np
import gc
import datetime
import glob
import glob

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 8000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
np.set_printoptions(suppress=False)

In [63]:
l = [pd.read_csv(filename, low_memory=False) for filename in glob.glob(r"C:\Users\****\Downloads\chain/*.txt")]
df = pd.concat(l, axis=0)

In [64]:
def seperate(data):
    data = data.rename(columns=lambda x: x.strip())
    # data.columns = [c.replace("]", "") for c in list(data.columns)]
    # data.columns = [c.replace("[", "") for c in list(data.columns)]
    data['[QUOTE_DATE]'] = pd.to_datetime(data['[QUOTE_DATE]'], format='%Y-%m-%d')
    data['[EXPIRE_DATE]'] = pd.to_datetime(data['[EXPIRE_DATE]'], format='%Y-%m-%d')
    data['[QUOTE_READTIME]'] = pd.to_datetime(data['[QUOTE_READTIME]'], format='%Y-%m-%d %H:%M')
   
    dfcall = data[['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
                   '[QUOTE_TIME_HOURS]','[UNDERLYING_LAST]', '[EXPIRE_DATE]','[EXPIRE_UNIX]', '[DTE]', '[STRIKE]','[STRIKE_DISTANCE]','[STRIKE_DISTANCE_PCT]',
                   '[C_IV]','[C_DELTA]','[C_GAMMA]','[C_VEGA]','[C_THETA]','[C_RHO]','[C_VOLUME]', '[C_LAST]', '[C_SIZE]','[C_BID]', '[C_ASK]']]
    dfcall['optype'] = 'C'
    dfcall.rename(columns={'[C_IV]': 'iv', '[C_DELTA]': 'delta', '[C_GAMMA]': 'gamma', '[C_VEGA]': 'vega', '[C_THETA]':'theta','[C_RHO]':'rho','[C_VOLUME]':'vol','[C_LAST]':'last','[C_SIZE]':'size','[C_BID]':'bid','[C_ASK]':'ask'}, inplace = True)
                 
                 
    dfput = data[['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]','[QUOTE_DATE]','[QUOTE_TIME_HOURS]','[UNDERLYING_LAST]', '[EXPIRE_DATE]','[EXPIRE_UNIX]', '[DTE]', '[STRIKE]','[STRIKE_DISTANCE]','[STRIKE_DISTANCE_PCT]',
                    '[P_IV]','[P_DELTA]','[P_GAMMA]','[P_VEGA]','[P_THETA]','[P_RHO]','[P_VOLUME]','[P_LAST]', '[P_SIZE]','[P_BID]','[P_ASK]']]
   
    dfput.rename(columns={'[P_IV]': 'iv', '[P_DELTA]': 'delta', '[P_GAMMA]': 'gamma', '[P_VEGA]': 'vega', '[P_THETA]':'theta','[P_RHO]':'rho','[P_VOLUME]':'vol','[P_LAST]':'last','[P_SIZE]':'size','[P_BID]':'bid','[P_ASK]':'ask'}, inplace = True)

    dfput['optype'] = 'P'
    dfchain = pd.concat([dfcall, dfput], axis=0)
    # UNIXTIME To HKT
    dfchain['[QUOTE_UNIXTIME]'] = pd.to_datetime(dfchain['[QUOTE_UNIXTIME]'], unit='s',utc=True).dt.tz_convert('HongKong')
    dfchain['[QUOTE_UNIXTIME]'] = dfchain['[QUOTE_UNIXTIME]'].dt.tz_localize(None)
    # Expire_UNIX
    dfchain['[EXPIRE_UNIX]'] = pd.to_datetime(dfchain['[EXPIRE_UNIX]'], unit='s',utc=True).dt.tz_convert('HongKong')
    dfchain['[EXPIRE_UNIX]'] = dfchain['[EXPIRE_UNIX]'].dt.tz_localize(None)
    dfchain.drop(columns={'[QUOTE_TIME_HOURS]','[QUOTE_DATE]'}, inplace = True)
    dfchain['[DTE]'] = dfchain['[DTE]'].astype(np.int16)
    dfchain['[STRIKE]'] = dfchain['[STRIKE]'].astype(np.int32)
    dfchain.columns = [c.replace("]", "") for c in list(dfchain.columns)]
    dfchain.columns = [c.replace("[", "") for c in list(dfchain.columns)]
    dfchain.rename(columns={'QUOTE_UNIXTIME': 'Quote_Date_HKT', '[QUOTE_READTIME]': 'Date_US',
                           'EXPIRE_UNIX': 'Expire_Date_HKT','EXPIRE_DATE':'Expire_Date_US',
                           'QUOTE_READTIME':'Quote_Date_US','UNDERLYING_LAST':'SPX_Last','STRIKE':'Strike','STRIKE_DISTANCE':'Distance','STRIKE_DISTANCE_PCT':'Distance_Pct'}, inplace = True)
    dfchain['iv'] = pd.to_numeric(dfchain['iv'], errors = 'coerce')
    dfchain['delta'] = pd.to_numeric(dfchain['delta'], errors = 'coerce')
    dfchain['gamma'] = pd.to_numeric(dfchain['gamma'], errors = 'coerce')
    dfchain['vega'] = pd.to_numeric(dfchain['vega'], errors = 'coerce')
    dfchain['vega'] = pd.to_numeric(dfchain['vega'], errors = 'coerce')
    dfchain['theta'] = pd.to_numeric(dfchain['theta'], errors = 'coerce')
    dfchain['rho'] = pd.to_numeric(dfchain['rho'], errors = 'coerce')
    dfchain['vol'] = pd.to_numeric(dfchain['vol'], errors = 'coerce')
    dfchain['last'] = pd.to_numeric(dfchain['last'], errors = 'coerce').astype(np.float16)
    dfchain['bid'] = pd.to_numeric(dfchain['bid'], errors = 'coerce').astype(np.float16)
    dfchain['ask'] = pd.to_numeric(dfchain['ask'], errors = 'coerce').astype(np.float16)
    dfchain['Distance'] = dfchain['Distance'].astype(np.float16)
    dfchain['Distance_Pct'] = dfchain['Distance_Pct'].astype(np.float16)
    dfchain['Contract_ID'] = "SPX"+dfchain['Expire_Date_US'].dt.strftime('%Y%m%d')+dfchain['optype']+dfchain['Strike'].astype(str)
    dfchain['mid'] = (dfchain['bid'] + dfchain['ask']) / 2
    dfchain['iv'] = np.round(100 * dfchain['iv'], 2)

    dfchain = dfchain[['Contract_ID','Quote_Date_HKT','Expire_Date_HKT','Quote_Date_US','Expire_Date_US','SPX_Last','optype','Strike','DTE','Distance', 'Distance_Pct', 'iv', 'delta', 'gamma', 'vega', 'theta', 'rho', 'vol', 'last', 'size', 'bid','mid','ask']]
    dfchain  = dfchain.sort_values(['Quote_Date_HKT','DTE','optype','Strike'],ascending=[True,True,True,True])
    
    # df = df[df['[STRIKE_DISTANCE]']<2500]    
    # dfchain.drop(columns={'[QUOTE_UNIXTIME]','[QUOTE_TIME_HOURS]','[EXPIRE_UNIX]'}, inplace = True)
    return dfchain

In [65]:
df = seperate(df)

In [97]:
df.head()

,Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,optype,Strike,DTE,Distance,Distance_Pct,iv,delta,gamma,vega,theta,rho,vol,last,size,bid,mid,ask,Weekday
0,SPX20220103C1200,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1200,0,3596.00,0.75,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3582.00,3592.00,3604.00,1
1,SPX20220103C1400,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1400,0,3396.00,0.71,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3382.00,3392.00,3404.00,1
2,SPX20220103C1600,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1600,0,3196.00,0.67,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3182.00,3192.00,3204.00,1
3,SPX20220103C1800,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1800,0,2996.00,0.62,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,2982.00,2992.00,3004.00,1
4,SPX20220103C1900,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1900,0,2896.00,0.60,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,2882.00,2892.00,2904.00,1


In [98]:
df['Weekday'] = df['Quote_Date_HKT'].dt.dayofweek

In [122]:
openday = df.loc[(df['Weekday']==3) &(df['DTE']==7) & (df['optype'] == 'P')]
openday = openday[openday['Distance']<1000]
openday = openday.loc[(openday['delta']>= -0.7) & (openday['delta']<= -0.69)]
closeday = df.loc[(df['Weekday']==2) &(df['DTE']<= 2)& (df['optype'] == 'P')]

In [123]:
openday.head()

,Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,optype,Strike,DTE,Distance,Distance_Pct,iv,delta,gamma,vega,theta,rho,vol,last,size,bid,mid,ask,Weekday
19781,SPX20220112P4745,2022-01-06 05:00:00,2022-01-13 05:00:00,2022-01-05 16:00:00,2022-01-12,4700.64,P,4745,7,44.41,0.01,12.61,-0.69,0.00,2.41,-1.99,-0.65,134.00,60.69,4 x 4,59.91,60.94,62.00,3
154073,SPX20220202P4395,2022-01-27 05:00:00,2022-02-03 05:00:00,2022-01-26 16:00:00,2022-02-02,4347.26,P,4395,7,47.69,0.01,21.03,-0.70,0.00,2.17,-5.35,-0.66,9.00,104.31,1 x 1,96.12,97.50,98.81,3
9416,SPX20220209P4580,2022-02-03 05:00:00,2022-02-10 05:00:00,2022-02-02 16:00:00,2022-02-09,4587.73,P,4580,7,7.70,0.00,8.63,-0.69,0.01,2.31,-4.71,-0.68,65.00,38.78,10 x 2,40.69,41.00,41.31,3
92602,SPX20220223P4515,2022-02-17 05:00:00,2022-02-24 05:00:00,2022-02-16 16:00:00,2022-02-23,4475.12,P,4515,7,39.91,0.01,13.59,-0.69,0.00,2.29,-2.54,-0.67,40.00,90.44,6 x 6,62.09,62.44,62.81,3
132466,SPX20220302P4310,2022-02-24 05:00:00,2022-03-03 05:00:00,2022-02-23 16:00:00,2022-03-02,4226.79,P,4310,7,83.19,0.02,25.45,-0.69,0.00,2.16,-3.62,-0.59,9.00,107.56,1 x 1,106.88,113.00,119.19,3


In [124]:
openday[0:100]

,Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,optype,Strike,DTE,Distance,Distance_Pct,iv,delta,gamma,vega,theta,rho,vol,last,size,bid,mid,ask,Weekday
19781,SPX20220112P4745,2022-01-06 05:00:00,2022-01-13 05:00:00,2022-01-05 16:00:00,2022-01-12,4700.64,P,4745,7,44.41,0.01,12.61,-0.69,0.00,2.41,-1.99,-0.65,134.00,60.69,4 x 4,59.91,60.94,62.00,3
154073,SPX20220202P4395,2022-01-27 05:00:00,2022-02-03 05:00:00,2022-01-26 16:00:00,2022-02-02,4347.26,P,4395,7,47.69,0.01,21.03,-0.70,0.00,2.17,-5.35,-0.66,9.00,104.31,1 x 1,96.12,97.50,98.81,3
9416,SPX20220209P4580,2022-02-03 05:00:00,2022-02-10 05:00:00,2022-02-02 16:00:00,2022-02-09,4587.73,P,4580,7,7.70,0.00,8.63,-0.69,0.01,2.31,-4.71,-0.68,65.00,38.78,10 x 2,40.69,41.00,41.31,3
92602,SPX20220223P4515,2022-02-17 05:00:00,2022-02-24 05:00:00,2022-02-16 16:00:00,2022-02-23,4475.12,P,4515,7,39.91,0.01,13.59,-0.69,0.00,2.29,-2.54,-0.67,40.00,90.44,6 x 6,62.09,62.44,62.81,3
132466,SPX20220302P4310,2022-02-24 05:00:00,2022-03-03 05:00:00,2022-02-23 16:00:00,2022-03-02,4226.79,P,4310,7,83.19,0.02,25.45,-0.69,0.00,2.16,-3.62,-0.59,9.00,107.56,1 x 1,106.88,113.00,119.19,3
8707,SPX20220309P4445,2022-03-03 05:00:00,2022-03-10 05:00:00,2022-03-02 16:00:00,2022-03-09,4385.62,P,4445,7,59.41,0.01,20.08,-0.69,0.00,2.23,-3.74,-0.66,1.00,106.00,36 x 36,90.69,92.25,93.81,3
129621,SPX20220330P4510,2022-03-24 04:00:00,2022-03-31 04:00:00,2022-03-23 16:00:00,2022-03-30,4455.84,P,4510,7,54.19,0.01,16.57,-0.69,0.00,2.27,-2.69,-0.67,16.00,59.12,3 x 3,75.81,77.00,78.19,3
23720,SPX20220413P4500,2022-04-07 04:00:00,2022-04-14 04:00:00,2022-04-06 16:00:00,2022-04-13,4480.60,P,4500,7,19.41,0.00,10.78,-0.70,0.00,2.25,-3.29,-0.67,173.00,56.25,7 x 7,50.50,50.91,51.31,3
62008,SPX20220420P4495,2022-04-14 04:00:00,2022-04-21 04:00:00,2022-04-13 16:00:00,2022-04-20,4446.84,P,4495,7,48.19,0.01,13.96,-0.70,0.00,2.26,-2.03,-0.61,4.00,68.38,2 x 5,63.50,64.75,66.00,3
99514,SPX20220427P4510,2022-04-21 04:00:00,2022-04-28 04:00:00,2022-04-20 16:00:00,2022-04-27,4459.55,P,4510,7,50.41,0.01,14.84,-0.69,0.00,2.28,-2.18,-0.61,1.00,94.69,100 x 100,66.31,68.50,70.62,3


In [139]:
openday.loc[openday.groupby("Quote_Date_HKT")["delta"].idxmin()]

,Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,optype,Strike,DTE,Distance,Distance_Pct,iv,delta,gamma,vega,theta,rho,vol,last,size,bid,mid,ask,Weekday
19781,SPX20220112P4745,2022-01-06 05:00:00,2022-01-13 05:00:00,2022-01-05 16:00:00,2022-01-12,4700.64,P,4745,7,44.41,0.01,12.61,-0.69,0.00,2.41,-1.99,-0.65,134.00,60.69,4 x 4,59.91,60.94,62.00,3
154073,SPX20220202P4395,2022-01-27 05:00:00,2022-02-03 05:00:00,2022-01-26 16:00:00,2022-02-02,4347.26,P,4395,7,47.69,0.01,21.03,-0.70,0.00,2.17,-5.35,-0.66,9.00,104.31,1 x 1,96.12,97.50,98.81,3
9416,SPX20220209P4580,2022-02-03 05:00:00,2022-02-10 05:00:00,2022-02-02 16:00:00,2022-02-09,4587.73,P,4580,7,7.70,0.00,8.63,-0.69,0.01,2.31,-4.71,-0.68,65.00,38.78,10 x 2,40.69,41.00,41.31,3
92602,SPX20220223P4515,2022-02-17 05:00:00,2022-02-24 05:00:00,2022-02-16 16:00:00,2022-02-23,4475.12,P,4515,7,39.91,0.01,13.59,-0.69,0.00,2.29,-2.54,-0.67,40.00,90.44,6 x 6,62.09,62.44,62.81,3
132466,SPX20220302P4310,2022-02-24 05:00:00,2022-03-03 05:00:00,2022-02-23 16:00:00,2022-03-02,4226.79,P,4310,7,83.19,0.02,25.45,-0.69,0.00,2.16,-3.62,-0.59,9.00,107.56,1 x 1,106.88,113.00,119.19,3
8707,SPX20220309P4445,2022-03-03 05:00:00,2022-03-10 05:00:00,2022-03-02 16:00:00,2022-03-09,4385.62,P,4445,7,59.41,0.01,20.08,-0.69,0.00,2.23,-3.74,-0.66,1.00,106.00,36 x 36,90.69,92.25,93.81,3
129621,SPX20220330P4510,2022-03-24 04:00:00,2022-03-31 04:00:00,2022-03-23 16:00:00,2022-03-30,4455.84,P,4510,7,54.19,0.01,16.57,-0.69,0.00,2.27,-2.69,-0.67,16.00,59.12,3 x 3,75.81,77.00,78.19,3
23720,SPX20220413P4500,2022-04-07 04:00:00,2022-04-14 04:00:00,2022-04-06 16:00:00,2022-04-13,4480.60,P,4500,7,19.41,0.00,10.78,-0.70,0.00,2.25,-3.29,-0.67,173.00,56.25,7 x 7,50.50,50.91,51.31,3
62008,SPX20220420P4495,2022-04-14 04:00:00,2022-04-21 04:00:00,2022-04-13 16:00:00,2022-04-20,4446.84,P,4495,7,48.19,0.01,13.96,-0.70,0.00,2.26,-2.03,-0.61,4.00,68.38,2 x 5,63.50,64.75,66.00,3
99514,SPX20220427P4510,2022-04-21 04:00:00,2022-04-28 04:00:00,2022-04-20 16:00:00,2022-04-27,4459.55,P,4510,7,50.41,0.01,14.84,-0.69,0.00,2.28,-2.18,-0.61,1.00,94.69,100 x 100,66.31,68.50,70.62,3


In [140]:
openday.shape

(22, 24)

In [183]:
openday = df.loc[(df['Weekday']==3) &(df['DTE']==7) & (df['optype'] == 'P')]
openday = openday[openday['Distance']<1000]
openday = openday.loc[(openday['delta']>= -0.7) & (openday['delta']<= -0.69)]
openday_short_leg = openday.loc[openday.groupby("Quote_Date_HKT")["delta"].idxmin()]
target = list(set(openday_short_leg['Contract_ID'].to_list()))
closeday = df[df['Contract_ID'].isin(target)]
closeday_short_leg = closeday.loc[(closeday['DTE']<= 6)& (closeday['DTE'] >=0)]
# Short Leg Size Refine
short_leg = pd.concat([openday_short_leg,closeday_short_leg], axis=0)
short_leg  = short_leg.sort_values(['Quote_Date_HKT','DTE','optype','Strike'],ascending=[True,True,True,True])
short_leg.columns = [f'sleg_{i}' if i not in ['Quote_Date_HKT','Quote_Date_HKT','Expire_Date_HKT','Quote_Date_US','Expire_Date_US','SPX_Last','Weekday'] else f'{i}' for i in short_leg.columns]

In [184]:
print(openday_short_leg.shape)
print(closeday_short_leg.shape)

(22, 24)
(105, 24)


In [185]:
short_leg[0:20]

,sleg_Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,sleg_optype,sleg_Strike,sleg_DTE,sleg_Distance,sleg_Distance_Pct,sleg_iv,sleg_delta,sleg_gamma,sleg_vega,sleg_theta,sleg_rho,sleg_vol,sleg_last,sleg_size,sleg_bid,sleg_mid,sleg_ask,Weekday
19781,SPX20220112P4745,2022-01-06 05:00:00,2022-01-13 05:00:00,2022-01-05 16:00:00,2022-01-12,4700.64,P,4745,7,44.41,0.01,12.61,-0.69,0.00,2.41,-1.99,-0.65,134.00,60.69,4 x 4,59.91,60.94,62.00,3
28825,SPX20220112P4745,2022-01-07 05:00:00,2022-01-13 05:00:00,2022-01-06 16:00:00,2022-01-12,4696.25,P,4745,6,48.81,0.01,11.98,-0.73,0.00,2.10,-1.88,-0.58,45.00,53.25,4 x 4,60.19,60.81,61.41,4
38004,SPX20220112P4745,2022-01-08 05:00:00,2022-01-13 05:00:00,2022-01-07 16:00:00,2022-01-12,4676.41,P,4745,5,68.62,0.01,9.99,-0.87,0.00,1.21,-1.01,-0.50,10.00,63.81,100 x 100,69.62,72.00,74.38,5
46966,SPX20220112P4745,2022-01-11 05:00:00,2022-01-13 05:00:00,2022-01-10 16:00:00,2022-01-12,4669.85,P,4745,2,75.12,0.02,13.33,-0.91,0.00,0.64,-1.41,-0.22,200.00,78.56,2 x 2,76.12,77.06,78.00,1
55766,SPX20220112P4745,2022-01-12 05:00:00,2022-01-13 05:00:00,2022-01-11 16:00:00,2022-01-12,4713.53,P,4745,1,31.50,0.01,10.49,-0.82,0.01,0.86,-2.54,-0.16,51.00,37.91,3 x 3,34.09,34.75,35.41,2
64685,SPX20220112P4745,2022-01-13 05:00:00,2022-01-13 05:00:00,2022-01-12 16:00:00,2022-01-12,4726.55,P,4745,0,18.41,0.00,0.26,-1.00,0.00,0.00,-3.83,-0.09,932.00,12.75,1 x 1,20.30,21.00,21.70,3
154073,SPX20220202P4395,2022-01-27 05:00:00,2022-02-03 05:00:00,2022-01-26 16:00:00,2022-02-02,4347.26,P,4395,7,47.69,0.01,21.03,-0.70,0.00,2.17,-5.35,-0.66,9.00,104.31,1 x 1,96.12,97.50,98.81,3
162675,SPX20220202P4395,2022-01-28 05:00:00,2022-02-03 05:00:00,2022-01-27 16:00:00,2022-02-02,4325.89,P,4395,6,69.12,0.02,28.08,-0.65,0.00,2.17,-4.65,-0.51,14.00,92.00,6 x 6,105.88,106.38,106.81,4
171412,SPX20220202P4395,2022-01-29 05:00:00,2022-02-03 05:00:00,2022-01-28 16:00:00,2022-02-02,4431.80,P,4395,5,36.81,0.01,22.89,-0.41,0.00,2.16,-5.28,-0.29,14.00,42.66,10 x 5,36.19,38.56,40.91,5
180002,SPX20220202P4395,2022-02-01 05:00:00,2022-02-03 05:00:00,2022-01-31 16:00:00,2022-02-02,4516.89,P,4395,2,121.88,0.03,22.86,-0.12,0.00,0.78,-3.56,-0.04,521.00,5.55,72 x 199,5.20,5.35,5.50,1


In [186]:
# Long Size
openday = df.loc[(df['Weekday']==3) &(df['DTE']==7) & (df['optype'] == 'P')]
openday = openday[openday['Distance']<1000]
openday = openday.loc[(openday['delta']>= -0.3) & (openday['delta']<= -0.29)]
openday_long_leg = openday.loc[openday.groupby("Quote_Date_HKT")["delta"].idxmax()]
target = list(set(openday_long_leg['Contract_ID'].to_list()))
closeday = df[df['Contract_ID'].isin(target)]
closeday_long_leg = closeday.loc[(closeday['DTE']<= 6)& (closeday['DTE'] >=0)]
# Short Leg Size Refine
long_leg = pd.concat([openday_long_leg,closeday_long_leg], axis=0)
long_leg  = long_leg.sort_values(['Quote_Date_HKT','DTE','optype','Strike'],ascending=[True,True,True,True])
long_leg.columns = [f'lleg_{i}' if i not in ['Quote_Date_HKT','Quote_Date_HKT','Expire_Date_HKT','Quote_Date_US','Expire_Date_US','SPX_Last','Weekday'] else f'{i}' for i in long_leg.columns]

In [190]:
long_leg.head()

,lleg_Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,lleg_optype,lleg_Strike,lleg_DTE,lleg_Distance,lleg_Distance_Pct,lleg_iv,lleg_delta,lleg_gamma,lleg_vega,lleg_theta,lleg_rho,lleg_vol,lleg_last,lleg_size,lleg_bid,lleg_mid,lleg_ask,Weekday
110075,SPX20220126P4455,2022-01-20 05:00:00,2022-01-27 05:00:00,2022-01-19 16:00:00,2022-01-26,4533.57,P,4455,7,78.62,0.02,21.74,-0.30,0.00,2.28,-3.48,-0.29,149.00,27.41,18 x 16,27.09,27.34,27.59,3
118609,SPX20220126P4455,2022-01-21 05:00:00,2022-01-27 05:00:00,2022-01-20 16:00:00,2022-01-26,4482.27,P,4455,6,27.30,0.01,17.03,-0.50,0.00,2.41,-5.23,-0.42,16.00,33.22,8 x 11,41.31,41.56,41.81,4
127333,SPX20220126P4455,2022-01-22 05:00:00,2022-01-27 05:00:00,2022-01-21 16:00:00,2022-01-26,4398.16,P,4455,5,56.81,0.01,23.49,-0.67,0.00,1.99,-4.66,-0.48,28.00,82.62,1 x 1,86.62,87.75,88.81,5
135881,SPX20220126P4455,2022-01-25 05:00:00,2022-01-27 05:00:00,2022-01-24 16:00:00,2022-01-26,4411.87,P,4455,2,43.09,0.01,33.52,-0.63,0.00,1.45,-9.73,-0.20,26.00,97.31,100 x 59,73.62,75.62,77.69,1
144434,SPX20220126P4455,2022-01-26 05:00:00,2022-01-27 05:00:00,2022-01-25 16:00:00,2022-01-26,4358.03,P,4455,1,97.00,0.02,0.11,-1.00,0.00,0.00,-9.59,-0.21,12.00,78.50,1 x 10,103.00,106.38,109.81,2


In [189]:
print(openday_long_leg.shape)
print(closeday_long_leg.shape)

(27, 24)
(130, 24)


In [192]:
df_merged = pd.merge(short_leg, long_leg, on=['Quote_Date_HKT', 'Expire_Date_HKT','Quote_Date_US','Expire_Date_US','SPX_Last','Weekday'])

In [198]:
df_merged.to_csv(r'C:\Users\ \Downloads\VIEW_EOD\Stra.csv', index = False)